In [1]:
import torch.nn as nn


class ResNet(nn.Module):
    def __init__(self, num_classes=2):
        super(ResNet, self).__init__()
        self.fc1 = nn.Linear(24, 96)
        self.bn1 = nn.BatchNorm1d(96)
        self.relu = nn.ReLU(inplace=True)
        self.fc2 = nn.Linear(96, 24)
        self.bn2 = nn.BatchNorm1d(24)
        self.fc3 = nn.Linear(24, num_classes)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        identity = x
        out = self.fc1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.bn2(out)
        out += identity
        out = self.relu(out)
        out = self.fc3(out)
        out = self.softmax(out)
        return out


In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('iot23_train_binary.csv')

In [4]:
X = df[['duration', 'orig_bytes', 'resp_bytes', 'missed_bytes', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'proto_icmp', 'proto_tcp', 'proto_udp', 'conn_state_OTH', 'conn_state_REJ', 'conn_state_RSTO', 'conn_state_RSTOS0', 'conn_state_RSTR', 'conn_state_RSTRH', 'conn_state_S0', 'conn_state_S1', 'conn_state_S2', 'conn_state_S3', 'conn_state_SF', 'conn_state_SH', 'conn_state_SHR']]

In [5]:
X.shape

(320000, 24)

In [6]:
Y = pd.get_dummies(df['label']).values

In [7]:
Y.shape

(320000, 2)

In [8]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X)
normalized_x = scaler.transform(X)
scaler.fit(Y)
normalized_y = scaler.transform(Y)

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
X_train, X_test, Y_train, Y_test = train_test_split(normalized_x, normalized_y, random_state=10, test_size=0.2)

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_classification

# Convert the data to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(Y_train, dtype=torch.float32)
X_val = torch.tensor(X_test, dtype=torch.float32)
y_val = torch.tensor(Y_test, dtype=torch.float32)

# X_train = X_train.clone().detach()
# y_train = y_train.clone().detach()
# X_val = X_val.clone().detach()
# y_val = y_val.clone().detach()

# Create the ResNet model
model = ResNet(num_classes=2)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 100
batch_size = 32
num_batches = len(X_train) // batch_size

for epoch in range(num_epochs):
    running_loss = 0.0
    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = (i + 1) * batch_size
        inputs = X_train[start_idx:end_idx]
        labels = y_train[start_idx:end_idx]
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
    epoch_loss = running_loss / num_batches
    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, epoch_loss))

# Evaluate the model on the validation set
with torch.no_grad():
    model.eval()
    outputs = model(X_val)
    _, predicted = torch.max(outputs.data, 1)
    total = y_val.size(0)
    _, normal_y_val = torch.max(y_val.data,1)
    correct = (predicted == normal_y_val).sum().item()
    accuracy = correct / total
    print('Accuracy on the validation set: {:.2f}%'.format(accuracy * 100))


Epoch [1/100], Loss: 0.5354
Epoch [2/100], Loss: 0.5283
Epoch [3/100], Loss: 0.5276
Epoch [4/100], Loss: 0.5274
Epoch [5/100], Loss: 0.5273
Epoch [6/100], Loss: 0.5271
Epoch [7/100], Loss: 0.5270
Epoch [8/100], Loss: 0.5270
Epoch [9/100], Loss: 0.5270
Epoch [10/100], Loss: 0.5270
Epoch [11/100], Loss: 0.5269
Epoch [12/100], Loss: 0.5269
Epoch [13/100], Loss: 0.5268
Epoch [14/100], Loss: 0.5270
Epoch [15/100], Loss: 0.5269
Epoch [16/100], Loss: 0.5269
Epoch [17/100], Loss: 0.5268
Epoch [18/100], Loss: 0.5268
Epoch [19/100], Loss: 0.5268
Epoch [20/100], Loss: 0.5267
Epoch [21/100], Loss: 0.5268
Epoch [22/100], Loss: 0.5267
Epoch [23/100], Loss: 0.5267
Epoch [24/100], Loss: 0.5267
Epoch [25/100], Loss: 0.5267
Epoch [26/100], Loss: 0.5267
Epoch [27/100], Loss: 0.5267
Epoch [28/100], Loss: 0.5268
Epoch [29/100], Loss: 0.5267
Epoch [30/100], Loss: 0.5267
Epoch [31/100], Loss: 0.5268
Epoch [32/100], Loss: 0.5267
Epoch [33/100], Loss: 0.5267
Epoch [34/100], Loss: 0.5267
Epoch [35/100], Loss: 0

In [12]:
torch.save(model,'resnet_binary.pt')